In [ ]:
## Importing packages

import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
## Reading data
dataRaw = pd.read_csv('./prices250.txt', delimiter = '\s+', header = None).T

## Adding column names (for each timepoint)
dataRaw.columns = ['price' + str(x) for x in range(1, dataRaw.shape[1]+1, 1)]

## Adding column representing which instrument
dataRaw['instrument'] = ['instrument' + str(x) for x in range(1, dataRaw.shape[0]+1, 1)]

## Converting to long format
data = pd.wide_to_long(dataRaw, stubnames = 'price', i = 'instrument', j = 'time')
data['instrument'] = [x[0] for x in data.index]
data['time'] = [x[1] for x in data.index]
data.reset_index(drop = True, inplace = True)

In [ ]:
## Autocorrelation graph

for i in range(1, 101, 1):
    instrument = data[data['instrument'] == ('instrument' + str(i))]['price']
    pd.plotting.autocorrelation_plot(instrument, linewidth = 0.5)

## Instrument 1 ARIMA

In [ ]:
## Subset for testing

dataInstrument1 = data[data["instrument"] == "instrument1"][["price", "time"]].set_index("time")
dataInstrument1.index = pd.to_datetime(dataInstrument1.index, unit = 'd')

In [ ]:
## Looking for optimal ARIMA parameters

# trainData = dataInstrument1["price"][:225]
# testData = dataInstrument1["price"][225:]

# p = range(30, 31, 1)
# d = q = range(0, 2)
# pdq = list(itertools.product(p, d, q))
# modelAIC = []

# for paramlist in pdq:
#     arimaModel = ARIMA(trainData, order = paramlist)
#     arimaModelFit = arimaModel.fit()
#     modelAIC.append(arimaModelFit.aic)

# for i in range(len(modelAIC)):
#     if list(modelAIC == min(modelAIC))[i] == True:
#         print(pdq[i])

In [ ]:
## Training using optimised parameters

trainData = dataInstrument1["price"][:225]
testData = dataInstrument1["price"][225:]
arimaModel = ARIMA(trainData, order = (30, 1, 0))
arimaModelFit = arimaModel.fit()
predictions = arimaModelFit.forecast(steps = 25)

testData.plot()
predictions.plot(color = "red")

## Modern Portfolio Theory Implementation

In [ ]:
dataRaw = open("./prices250.txt", "r")